<P> <img src="https://i.ibb.co/gyNf19D/nhslogo.png" alt="nhslogo" border="0" width="100" align="right"><font size="6"><b> CS5131 Introduction to Artificial Intelligence</b> </font>  

# Module Project

## TITLE
### _Ray Sim, Kayden Tan_

## Table of Contents
<a id='ToC'/>
On Jupyter<br>
1. <a href=#Importing>Importing</a><br>
2. <a href=#Functions>Functions</a><br>
3. <a href=#Autoencoders>Autoencoders</a><br>
4. <a href=#Training/Testing>Training/Testing</a><br>
On colab<br>
1. <a href=#scrollTo=eP-dRA9g4P0W&line=3&uniqifier=1>Importing</a><br>
2. 

## Importing
<a id = 'Importing'/>
<a href=#ToC>back</a><br>
<a href=#scrollTo=iSjP6Hfu3uFC&line=8&uniqifier=1>back(on colab)</a>

In [1]:
import torch, pretty_midi, matplotlib.pyplot as plt, numpy as np, pandas as pd, librosa.display, time
from os import listdir
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

## Functions
<a id = 'Functions'/>
<a href=#ToC>back</a><br>
<a href=#scrollTo=iSjP6Hfu3uFC&line=8&uniqifier=1>back(on colab)</a>

In [4]:
def fix(tensor_arr):
    arr = []
    numpy_beeg = tensor_arr.detach().cpu().numpy()
    for i in range(numpy_beeg.shape[0]):
        arr.append(numpy_beeg[i])
    
    thing = np.hstack(arr)
    return thing

In [5]:
def plot_piano_roll(pm, start_pitch = 2, end_pitch = 100, fs=100):
    librosa.display.specshow(pm.get_piano_roll(fs)[start_pitch:end_pitch],
                             hop_length=1, sr=fs, x_axis='time', y_axis='cqt_note',
                             fmin=pretty_midi.note_number_to_hz(start_pitch))

#plot_piano_roll(test_song[7])

In [6]:
def plot_tensor(item, start_pitch = 2, end_pitch = 100, fs = 100):
    librosa.display.specshow(fix(item), hop_length=1, sr=100, x_axis='time', y_axis='cqt_note', fmin=pretty_midi.note_number_to_hz(2))

#plot_tensor(test_arr[7])

In [7]:
def plot_autoencoded(model, item, start_pitch = 2, end_pitch = 100, fs = 100):
    plot_tensor(model.forward(item), start_pitch, end_pitch, fs)
#plot_autoencoded(model2, test_arr[7])

In [12]:
def split_song(song, time_interval = 2, fs = 100):
    #each stack is all the data each 2s interval
    #each row should represent each instrument
    #elements in each row should be the piano roll of each instrument in the 2s interval 
    arr = []

    roll = song.get_piano_roll(fs = fs)
    #print(roll.shape, end = '\r') #because it takes a while, to check if stuck
    
    for j in range(0, roll.shape[1], time_interval*fs): #splitting the columns into groups of 20
        sub_array = roll[:, j:j+time_interval*fs]
            
        if (sub_array.shape[1] == time_interval*fs):
            #if the shape of the subarray is (128, 20), then just stack
            arr.append(sub_array)
            
        else:            
            add_columns = time_interval*fs - sub_array.shape[1] #see how many columns are missing
            padding = np.zeros((128,add_columns)) #create the zero array to pad
            new = np.concatenate((sub_array, padding), axis = 1) #pad the array until there are 20 columns
            arr.append(new) #stack
    
    thing = np.stack(arr, axis = 0)
    return thing

def split_songs(songLst, time_interval = 2, fs = 100):
    thing_arr = []
    for i in songLst:
        thing_arr.append(torch.tensor(split_song(i, time_interval = 1, fs = 100)).cuda().float())

    train_arr, test_arr = train_test_split(thing_arr, test_size = 0.2, random_state = 43)
    train_song, test_song = train_test_split(songLst, test_size = 0.2, random_state = 43)
    return train_arr, test_arr, train_song, test_song


#train_arr, test_arr, train_song, test_song = split_songs(songLst)

In [13]:
def find_loss(model, test_arr):
    test_loss = 0
    criterion = nn.MSELoss()
    for data in test_arr:
        test_loss += criterion(model(data), data).item()*data.size(0)
    test_loss /= len(test_arr)
    return test_loss

In [15]:
def runModel(model, data_arr, n_epochs = 1000):    
    
    #initialisation
    encoder = model().cuda()
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(encoder.parameters(), lr=0.001)
    initial = time.time()
    
    for epoch in range(n_epochs):
        train_loss = 0 
    
        for data in data_arr: #for each song
            optimizer.zero_grad() 
            
            outputs = encoder.forward(data) #get the output after the encoder on the input
            loss = criterion(outputs, data) #MSE the difference
            loss.backward() #find the gradient of the loss compared to the current model params
            optimizer.step() #optimize the params
            #train_loss += loss.item()*data.size(0)
            
        train_loss /= len(data_arr) #get the average loss across songs
        print("Epoch: {} Train Loss: {} Time Elapsed: {}".format(epoch+1, -1, time.time() - initial), end = '\r')
    return encoder

#model = runModel(TestEncoder, train_arr, 100)

## Autoencoders
<a id = 'Autoencoders'/>
<a href=#ToC>back</a><br>
<a href=#scrollTo=iSjP6Hfu3uFC&line=8&uniqifier=1>back(on colab)</a>

In [19]:
# access weights of autoencoder using this https://discuss.pytorch.org/t/access-all-weights-of-a-model/77672
# recurrent example here https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
# autoencoder thing https://medium.com/pytorch/implementing-an-autoencoder-in-pytorch-19baa22647d1
# no recurrent yet

class TestEncoder(nn.Module): #where nn.Module is the parent
    
    def __init__(self): 
        #initialising nn.Module
        super(TestEncoder, self).__init__()
               
        #Encoding
        #nn.Conv1D(n_instruments_input, n_instruments_output, kernel_size)
        #padding is added on both sides
        self.conv1 = nn.Conv1d(128, 32, 10, padding = 4, stride = 2) #20 columns to 10
                
        #Pooling
#         self.pool = MaxPool2d(4, 4)
        
        #Decoding
        self.convT1 = nn.ConvTranspose1d(32, 128, 10, padding = 4, stride = 2)
    
    def forward(self, inpt):
        encoded = F.relu(self.conv1(inpt))
        decoded = F.relu(self.convT1(encoded))
        
        return decoded

In [20]:
class TestEncoder2(nn.Module): #where nn.Module is the parent
    
    def __init__(self):        
        #initialising nn.Module
        super(TestEncoder2, self).__init__()
               
        #Encoding
        #nn.Conv1D(n_instruments_input, n_instruments_output, kernel_size)
        #padding is added on both sides
        self.conv1 = nn.Conv1d(128, 64, 10, padding = 4, stride = 2)
        self.conv2 = nn.Conv1d(64, 32, 10, padding = 4, stride = 2)
        
        #Decoding
        self.convT2 = nn.ConvTranspose1d(32, 64, 10, padding = 4, stride = 2)
        self.convT1 = nn.ConvTranspose1d(64, 128, 10, padding = 4, stride = 2)
        
    
    def forward(self, inpt):
        encoded1 = F.relu(self.conv1(inpt))
        encoded2 = F.relu(self.conv2(encoded1))
        
        decoded1 = F.relu(self.convT2(encoded2))
        decoded2 = F.relu(self.convT1(decoded1))
        

        return decoded2

In [21]:
class TestEncoder3(nn.Module): #where nn.Module is the parent
    
    def __init__(self): 
        #initialising nn.Module
        super(TestEncoder3, self).__init__()
               
        #Encoding
        #nn.Conv1D(n_instruments_input, n_instruments_output, kernel_size)
        #padding is added on both sides
        self.conv1 = nn.Conv1d(128, 64, 10, padding = 4, stride = 2)
        self.conv2 = nn.Conv1d(64, 32, 10, padding = 4, stride = 2)
        
        #Decoding
        self.convT1 = nn.ConvTranspose1d(32, 64, 10, padding = 4, stride = 2)
        self.convT2 = nn.ConvTranspose1d(64, 128, 10, padding = 4, stride = 2)
        

    
    def forward(self, inpt):
        encoded = F.relu(self.conv1(inpt))
        encoded = F.relu(self.conv2(encoded))
        decoded = F.relu(self.convT1(encoded))
        decoded = F.relu(self.convT2(decoded))
        return decoded
    
    def forward2(self, inpt):
        encoded = F.relu(self.conv1(inpt))
        encoded = F.relu(self.conv2(encoded))
        return encoded

In [22]:
class TestEncoder4(nn.Module): #where nn.Module is the parent
    
    def __init__(self): 
        #initialising nn.Module
        super(TestEncoder4, self).__init__()
               
        #Encoding
        #nn.Conv1D(n_instruments_input, n_instruments_output, kernel_size)
        #padding is added on both sides
        self.conv1 = nn.Conv1d(128, 128, 10, padding = 4, stride = 2)
        self.conv2 = nn.Conv1d(128, 128, 10, padding = 4, stride = 2)
        
        #Decoding
        self.convT1 = nn.ConvTranspose1d(128, 128, 10, padding = 4, stride = 2)
        self.convT2 = nn.ConvTranspose1d(128, 128, 10, padding = 4, stride = 2)
        
    
    def forward(self, inpt):
        encoded = F.relu(self.conv1(inpt))
        encoded = F.relu(self.conv2(encoded))
        decoded = F.relu(self.convT1(encoded))
        decoded = F.relu(self.convT2(decoded))
        return decoded
    
    def forward2(self, inpt):
        encoded = F.relu(self.conv1(inpt))
        encoded = F.relu(self.conv2(encoded))
        return encoded

## Initialisation

In [23]:
songLst = [] 
for file in listdir('MiDi'):
    print('Importing {}...'.format(file))
    f = pretty_midi.PrettyMIDI('MiDi/'+file)
    songLst.append(f)
len(songLst)

Importing 3LAU feat. Bright Lights - How You Love Me  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Above & Beyond - Tri-state  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Above & Beyond - We're All We Need (Original Mix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing Above & Beyond feat. Alex Vargas - Blue Sky Action  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Adam F - Circles  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Aether & Enzalla - Elysia's Heart  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Afrojack & David Guetta - Another Life  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Alan Walker - Alone  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Alan Walker - Faded (Original Mix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing Alan Walker - Sing Me To Sleep  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Alan Walker - The Spectre  (midi by Carlo Prato) (www.cprato.com).mid...
Im

Importing Dawn Golden - All I Want (Manila Kill Remix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing DBSTF & Maurice West - Temple (Extended Mix) (Midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - 2448  (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - 4ware (Extended Mix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - Cat Thruster  (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - Deus Ex Machina  (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - Glish  (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - Hyperlandia  (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - No Problem  (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - So There I Was  (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 - Strobe (Dimension Club Mix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing deadmau5 feat. Grabbitz - Let Go  (midi by Carlo Pr

Importing Kygo & Ellie Goulding - First Time  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Kygo & Selena Gomez - It Ain't Me  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Kygo feat. Whitney Houston - Higher Love (Original Mix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing Labirinth - Last Time (Knife Party Remix) (midi By Carlo Prato) (www.cprato.com).mid...
Importing Lacuna - Celebrate The Summer (Cascada Remix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing Lady Gaga - Always Remember Us This Waya (midi by Carlo Prato) (www.cprato.com).mid...
Importing Lewis Capaldi - Before You Go_ (midi by Carlo Prato) (www.cprato.com).mid...
Importing Lil Nas X - Old Town Road  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Lil Nas X - Panini  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Lilly Wood & The Prick and Robin Schulz - Prayer In C (midi by Carlo Prato) (www.cprato.com).mid...
Importing LTN - Never Let Me Go (Original Mix) (midi

Importing SlyphStorm - Morning Mist  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Snatt & Vix - Revive (Airborn Remix) (Midi by Carlo Prato) (www.cprato.com).mid...
Importing Sonic Battle - Emerald Beach  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Soundflower - Back 2 U  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Starley - Call On Me (Ryan Riback Remix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing Steve Angello - Wasted Love (Grum Remix) (midi by Carlo Prato) (www.cprato.com).mid...
Importing Stian K - Nostalgy  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Streex - Alive  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Sunny Day Real Estate - The Ocean  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Suyano & Adventurer - Overload  (midi by Carlo Prato) (www.cprato.com).mid...
Importing Swedish House Mafia - Don't You Worry Child (midi by Carlo Prato) (www.cprato.com).mid...
Importing Taylor Swift - Look What Yo

348

In [24]:
train_arr, test_arr, train_song, test_song = split_songs(songLst)

AssertionError: Torch not compiled with CUDA enabled

## Training/Testing
<a id = 'Training/Testing'/>
<a href=#ToC>back</a><br>
<a href=#scrollTo=iSjP6Hfu3uFC&line=8&uniqifier=1>back(on colab)</a>